## Rotation in OpenCV — Two Main Ways

There are two primary ways to rotate an image in OpenCV:

1. **Using `cv::getRotationMatrix2D` + `cv::warpAffine`**
2. **Using `cv::remap` with a custom rotation mapping**

Both work the same way mathematically — they apply a **2×3 affine transform** that rotates pixels about a chosen **center point**.

---

## Coordinate System Recap

OpenCV uses the **image coordinate system**:

* Origin `(0,0)` is **top-left**.
* `x` increases **to the right**.
* `y` increases **downward**.



In a normal coordinate system `+θ` means rotation in counterclockwise, an arrow pointing from `x` axis toward `y` axis

```

y ↑
  |    
  |       ↰
(0,0) ----→ x
  
  ↓
  
```

So a **positive rotation angle θ** in OpenCV corresponds to a **clockwise** visual rotation.
This is opposite to the standard mathematical convention (where +θ is counterclockwise).

```
(0,0) ----→ x
  |      ↵
  |
  ↓
  y
```




---

## `cv::getRotationMatrix2D`

Signature:

```cpp
cv::Mat getRotationMatrix2D(cv::Point2f center, double angle, double scale)
```

Returns a 2×3 affine transform matrix **M** that combines:

* rotation by `angle` degrees around `center`
* optional uniform scaling by `scale`

---

## The Rotation Matrix

The resulting 2×3 matrix has the form:

$$
M =
\begin{bmatrix}
\alpha & \beta & (1 - \alpha) c_x - \beta c_y \\
-\beta & \alpha & \beta c_x + (1 - \alpha) c_y
\end{bmatrix}
$$

where

$$
\alpha = \text{scale} \cdot \cos\theta 
$$

$$
\beta = \text{scale} \cdot \sin\theta
$$

and $c_x, c_y$ is the rotation center.

---

#### Example (pure rotation, no scaling)

If `scale = 1`, then:

$$
M =
\begin{bmatrix}
\cos\theta & \sin\theta & (1 - \cos\theta)c_x - \sin\theta c_y \\
-\sin\theta & \cos\theta & \sin\theta c_x + (1 - \cos\theta)c_y
\end{bmatrix}
$$

---

####  The Reference Point (Center of Rotation)

The **center point** determines what stays fixed during rotation.

Typical choices:

* `(0,0)` → rotate around top-left corner
* `(cols/2, rows/2)` → rotate around image center

When you call:

```cpp
cv::Point2f center(img.cols/2.0f, img.rows/2.0f);
M = cv::getRotationMatrix2D(center, angle, 1.0);
```

it means **the image center remains fixed** during rotation.

---

####  Applying the Rotation

The rotation is applied using:

```cpp
cv::warpAffine(src, dst, M, cv::Size(cols, rows));
```

Internally, for each destination pixel `(u,v)`:

$$
\begin{bmatrix}
x \\ y \\ 1
\end{bmatrix}
=
M
\begin{bmatrix}
u \\ v \\ 1
\end{bmatrix}
$$

and then `dst(v,u) = src(y,x)` via interpolation.

Thus, **the transform maps coordinates from destination to source** (inverse mapping).

---

## **Numerical Example**

Rotate image, `+90` degree, once around the center of the image `(1,1)` and once around the top left `(0,0)` 
When you change the **rotation center**, OpenCV’s rotation matrix changes its **translation terms**, and you get different resulting pixel layouts.

---

####  Source Image


$$
I_s =
\begin{bmatrix}
10 & 20 & 30 \\
40 & 50 & 60 \\
70 & 80 & 90
\end{bmatrix}
$$

Coordinates (OpenCV convention):

| (x,y) | Value |
| ----- | ----- |
| (0,0) | 10    |
| (1,0) | 20    |
| (2,0) | 30    |
| (0,1) | 40    |
| (1,1) | 50    |
| (2,1) | 60    |
| (0,2) | 70    |
| (1,2) | 80    |
| (2,2) | 90    |

---

####  Case 1: Rotation about **Top-Left Corner (0,0)**

We use `center = (0,0)` and `angle = 90°` (OpenCV positive = clockwise).

**a) Rotation matrix**

From OpenCV:

$$
M =
\begin{bmatrix}
\cos\theta & \sin\theta & (1 - \cos\theta)c_x - \sin\theta c_y \\
-\sin\theta & \cos\theta & \sin\theta c_x + (1 - \cos\theta)c_y
\end{bmatrix}
$$

With $ c_x = 0,\\ c_y = 0,\\ \theta = 90° $:

$
\cos\theta = 0, \quad \sin\theta = 1
$

So:

$$
M =
\begin{bmatrix}
0 & 1 & 0 \\
-1 & 0 & 0
\end{bmatrix}
$$

---

**b) Apply to coordinates**

The transform is applied as:
$
\begin{bmatrix}x' \\ y'\end{bmatrix} =
\begin{bmatrix}0 & 1\\ -1 & 0\end{bmatrix}
\begin{bmatrix}x \\ y\end{bmatrix}
$

| Source (x,y) | New (x',y') | Value |
| ------------ | ----------- | ----- |
| (0,0)        | (0,0)       | 10    |
| (1,0)        | (0,-1)      | 20    |
| (2,0)        | (0,-2)      | 30    |
| (0,1)        | (1,0)       | 40    |
| (1,1)        | (1,-1)      | 50    |
| (2,1)        | (1,-2)      | 60    |
| (0,2)        | (2,0)       | 70    |
| (1,2)        | (2,-1)      | 80    |
| (2,2)        | (2,-2)      | 90    |

Because of negative (y') values, many pixels move **above the top** of the image — they get clipped unless you enlarge the canvas.



| (x′, y′) | value |
| -------- | ----- |
| (0,0)    | 10    |
| (1,0)    | 40    |
| (2,0)    | 70    |

Everything else is outside the image and filled by the border value (default = 0).

$$
I_d =
\begin{bmatrix}
10 & 40 & 70 \\
0 & 0 & 0 \\
0 & 0 & 0
\end{bmatrix}
$$

---

####  Case 2: Rotation about **Center (1,1)**

Now $c_x = 1, c_y = 1$, same `90°` rotation.


$
\cos\theta = 0,\quad \sin\theta = 1
$


---


$$
M =
\begin{bmatrix}
0 & 1 & (1-0)\cdot1 - 1\cdot1 = 0\\
-1 & 0 & 1\cdot1 + (1-0)\cdot1 = 2
\end{bmatrix}
=
\begin{bmatrix}
0 & 1 & 0\\
-1 & 0 & 2
\end{bmatrix}
$$

---

**Apply to coordinates**

$$
\begin{bmatrix}x' \\ y'\end{bmatrix}
=
\begin{bmatrix}
0 & 1 & 0\\
-1 & 0 & 2
\end{bmatrix}
\begin{bmatrix}x\\y\\1\end{bmatrix}
$$


| Source (x,y) | (x',y') | Value |
| ------------ | ------- | ----- |
| (0,0)        | (0,2)   | 10    |
| (1,0)        | (0,1)   | 20    |
| (2,0)        | (0,0)   | 30    |
| (0,1)        | (1,2)   | 40    |
| (1,1)        | (1,1)   | 50    |
| (2,1)        | (1,0)   | 60    |
| (0,2)        | (2,2)   | 70    |
| (1,2)        | (2,1)   | 80    |
| (2,2)        | (2,0)   | 90    |

 All transformed coordinates are within `[0,2]×[0,2]`.
So the output image fits perfectly in its 3×3 frame.

---

**Visual Result (Case 2)**

$$
I_d =
\begin{bmatrix}
30 & 60 & 90 \\
20 & 50 & 80 \\
10 & 40 & 70
\end{bmatrix}
$$

That is a **90° clockwise rotation about the center**.

---



In [3]:
import cv2
import numpy as np
import math

src = np.array([[10,20,30],
                [40,50,60],
                [70,80,90]], dtype=np.uint8)

# Case 1: rotate about (0,0)
M1 = cv2.getRotationMatrix2D((0,0), 90, 1)
dst1 = cv2.warpAffine(src, M1, (3,3))

# Case 2: rotate about center (1,1)
M2 = cv2.getRotationMatrix2D((1,1), 90, 1)
dst2 = cv2.warpAffine(src, M2, (3,3))

print("M1:\n", M1)
print("M2:\n", M2)
print("dst1 (center=0,0):\n", dst1)
print("dst2 (center=1,1):\n", dst2)


M1:
 [[ 6.123234e-17  1.000000e+00  0.000000e+00]
 [-1.000000e+00  6.123234e-17  0.000000e+00]]
M2:
 [[ 6.12323400e-17  1.00000000e+00 -1.11022302e-16]
 [-1.00000000e+00  6.12323400e-17  2.00000000e+00]]
dst1 (center=0,0):
 [[10 40 70]
 [ 0  0  0]
 [ 0  0  0]]
dst2 (center=1,1):
 [[30 60 90]
 [20 50 80]
 [10 40 70]]
